In [145]:
import requests
import jwt
import time
import secrets
import json
from jwcrypto import jwk
baseUrl = "http://localhost:8080/iis"
tenant = "Connectathon"
fhirUrl = baseUrl + "/fhir/" + tenant
session = requests.Session()
headers={"Content-Type": "application/json"}
alg = "RS384"
config = session.get(baseUrl + "/.well-known/smart-configuration")
# print(config.text)
# Use following command to generate private key
# ssh-keygen -t rsa -b 4096 -E SHA384 -m PEM -P "" -f RS384.key
with open("RS384.key", "rb") as pemfile: 
    key = jwk.JWK.from_pem(pemfile.read())
print(key.export_public())
keys= [key.export_public(),
       '''{"kty":"EC","crv":"P-384","x":"9woYwaz77oNH2KLuEgQoYw8sfdox3G90AEx3bp3XGcAjFSb1SMNSQvCzOFc5Rza-","y":"vpC_ySRlrpghxXtn89HcWb9gf1ecfMqjv19-XKgAyBLtDZbP4tPP6PVUbo7WwfjO","use":"sig","alg":"ES384","kid":"0kIAOUPvagpQ7kqnIkpB0M3ay0+ZZFPn91P6jtcEAZo="}''',
       '''{"kty": "RSA","e": "AQAB","kid": "MDJSMVLPm-PIbhTPs4Nb265mNLXR4hoZ3Gezu13bOj8","n": "qfXsRgkLUk34OgOSlLyI1CB_TPdSLltAFJMq3hAvEdnU0sJsbyA4sgr2x_8WILa1gmeaz1JaNrPjEn0kHbzIfCyLa_kN2iSyLyayleYB-XOdrHDmU5dzzn6f8UZILrkZ3cbtQTa5FQ0DJunIy29H1yLUlro3kYKciKcJve8AU4byqFXOTRfZnxja8xHsrDgeSfes5bVju8rh2K4Qp0fdqofPPsZls_mv0Or_-eL_vuoOk4bYWPgBWrAA11EJrBu8K8p93BdH2YVznNofczhyneomOlX5amK34eCimAIXqAevf5_gvVm3IXV0dpvhmlgHVg6rtRnyIFnojbCG6DEwbw"}''',
       '''{"kty": "RSA", "alg": "RS384", "n": "t15JOq_cmMFzn4bB8nHFtVMkCcO6hX6dTdnXJ7lEVcqehINYqYxdsYUAzybh69DSB0P9kn7VZit6v5uSiB2TZ_B1XF1V5ZP5Z1nVSp3CjVHXwb7sPT5uU-xXaenohnNnR_W4DwPbqwfN4BA9nPnxQYF0eXQpba8Ntc0wxxLyWaKHi5rULyJ4ZgT_mQwJjceTtGnZA6wYi00Sbi8pe2LRS9a9vy0Rrcg4LRp6xycZSRVDyrfk8rc8ZbT43RKrp8jK5FzK9-_hs9bOM66FawvJVIE3MqgsSqMyY-CYJ5Gi7dfojigYQZd5iX0sdKO_ZCH-px5gUonKqabZm25ZGVyOlQ", "e": "AQAB", "key_ops": ["verify"], "ext": true, "kid": "e2581e6b57fbbb69dcf29867d2a926b7" }''',
        '''{"kty":"RSA","e":"AQAB","kid":"z9ueUVrwGj","key_ops":["verify"],"alg":"RS384","n":"lK3hzJx9KLB3dThx_T2CKmX555ZVuzwjeMVSluhJ1vyWMucsQupey7gqr6Z_QFWDeEAlb19lHAVSjcRtt7a7YfYTmRsBECw0M61d25nNepvP6JhcT9wG0WmkNcEJWtocqzZd5hn8W9zONdjGq1FbgUVZm0jYcVVIeR6keU0bYPQhrFFYmoVlJIAwH7jAaf8ZXgdQgc25gRh5HFzgsO1aeFROJo-dQnh7yDWaThMQcgf0siSGLGRZp3jih1y7oqC9a8XVQeWS-awBIMatAjKFRweQOvDDJS4gJGh2w4HOBmLw_52iEuQ2ocrhxad2Nq5kZyivTnmjx2PaRpgUMkBebQ"}''',
        '''{ "kty": "RSA","n": "qOG6V6hG-YiO98yX3kyXLtEu2qWgXRY-1UJfWvTDQ62fmWYRAmPVX9xCCNTc-f7PidUZ6dNoOEulhW3VNicWEcT-x5hKXO7jCge2K4_gCmkLJKzx04yJbutiW1zpSYIyZk1xNv5Hljf2TOeeRSYWO7mGUQ-xzip7Xh68OXOsy5IhxQekOQazRmC2Zk1KEx-Af9k6qtEbK32Z8VJffGDdMxB1jgp5PSfigZudK3tRi6LVSlxfYfd_OlNB3YrNtSUKkcMiosYLIaaVDBnon7I9Gd8FJwdVwrvaS5IuA3vSNxltfIMF7pJm2wyLMi4E7MrcWDWxk804NnazFctMJJvekQ","e": "AQAB","use": "sig","alg": "RS384","kid": "fizCOwjuzhM4/l4h5li3L643yWiKFSQ4w+SKbyb4ig4="}''',
        '''{"kty": "RSA", "n": "2NLnivBeuDWhi6MwCkCk4vgU42JZ3La4z024ssGUmvz4IF5-9D5uHeiS0nZpOabdO90oq5_w-vxlDbdO9IsaZ9KAN_jh9lPY7HqqeTsk7XkQksLhi0Uzj1LrHFeULSpIsu2Fz9clvE9vLt8PhBJmPFwzCUoOWdyDeD4iGYr007zOMMpAYC--QiO7_QUY8fOBEyA3Sh0X9bbjh9eCGpSZI2vKUeW1f-EsqOD53YS7d5Z2POOYweVLGTf9k7QKpdwJFCQ_7Nh6vCOQqdmOY5WgXG-MgyhynKKGJN8QIY8fIDOOwSRj75fAF-d9F1jmY90LAYJukJx0qjtLfc8XH4W7vw","e": "AQAB","ext": true, "kid": "f6c17f82aad11fcc52e052a35cca", "alg": "RS256", "use": "sig"}''',
    ]
for webkey in keys:
    register = session.post(f"{baseUrl}/registerClient", data=webkey)
    # print(webkey,"")
    print(register.status_code)

{"e":"AQAB","kid":"jWVi44uPpn9Xblv5LztrynJ8jNMU-zi_cOD2A7RL25w","kty":"RSA","n":"uW32CUHy3IyvySdEpGtof61pml-ygKnxq_OVAAon2i_orKw4rmJ4siqSnCK3nbwYeHSikNXd-AsG1buBLp7GgVjZq6JRRcpkWQO0RX7UygCKLrx0wUpyCq5FCY8FBt2-VXaZD52VclbTmwbaFoggpuLh3dlpwj0gQmnXx7CgsyR40X7nkUAY4FHkSyTTDmq4GQKjOxh2T9KSE29fx1viw7IQYiVWWFfjKY30EZV0oAsOB2Pywq_1qLfYKvHctGaVkCV4UkwDPcp-Y-fLwYYJD8EJNHr6mR-cxFTYRMAnDq-HG8KSYKLzjq6MHjrh1VwQWHF1x_aqXr7FdIM7cAm4Vo6NdhgZaLE-5vgsGd1j_gRqKt_0duSjtBOu4FMCW7Ban062MEdDX96TiuehrgUFyTF0N3lM7qliOoqY753IsEIUzIDmOq9qxU1kDRI1GXbhL1PsZREmDI4dqDzLHgnSVY5o4NIAU86FmyfyY15wilbOsCWTKBMpbu1zb7MXOOsV1X03DzgRDVRiL0KGyoN9rAnTAZ8bc3bsZW1p6bfI4gx4Z1XvksPYXJN_w0eCwo34U8rq2tmOmWEH-Dd7PsedDSrlsH0QyOpVccFX8J_5_XIBTDBm4sp9vzIVY6IciVXof7ZNeyIyi4xEoUgt7rODpE5DrOt53qrH8P1qxBXtZok"}
200
200
200
200
200
200
200


In [142]:
with open("RS384.key", "rb") as keyfile: 
    secretKey = keyfile.read()
with open("RS384.key.pub", "rb") as keyfilepub: 
    publicKey = keyfilepub.read()
tokenUrl = f"{baseUrl}/token"

jwt_claims = {"iss": tokenUrl,"sub": tokenUrl, "aud": tokenUrl, "exp": int(time.time()*1000) + 30000, "jti": secrets.token_urlsafe()}
jwt_headers = {"alg":alg, "kid":key.get("kid"), "typ":"JWT"}

encoded_jwt = jwt.encode(payload=jwt_claims, key=secretKey, algorithm=alg, headers=jwt_headers)
# print(encoded_jwt)
decode = jwt.decode(encoded_jwt, publicKey, algorithms=[alg],audience=tokenUrl)
# print(decode)
# login = session.post(f"{baseUrl}/token", json={"client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer", "client_assertion": encoded_jwt})
login = session.get(tokenUrl, params={"client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer", "client_assertion": encoded_jwt, "grant_type": "client_credentials", "scope": 'system/*.rs'})
print(login.status_code)
token = json.loads(login.text)["access_token"]

200


In [129]:
result = session.get(f"{fhirUrl}/Binary", headers={"Authorization": f"Bearer {token}"})
print(result, result.content)


<Response [200]> b'{\n  "resourceType": "Bundle",\n  "id": "10f0874e-370f-4561-8ff5-4092569c7537",\n  "meta": {\n    "lastUpdated": "2024-01-17T14:21:38.489-05:00"\n  },\n  "type": "searchset",\n  "total": 99,\n  "link": [ {\n    "relation": "self",\n    "url": "http://localhost:8080/iis/fhir/Connectathon/Binary"\n  } ],\n  "entry": [ {\n    "fullUrl": "http://localhost:8080/iis/fhir/Connectathon/Binary/qOsQWiWzw1sXbo2EKB8OMqIMNRnsXVpS",\n    "resource": {\n      "resourceType": "Binary",\n      "id": "qOsQWiWzw1sXbo2EKB8OMqIMNRnsXVpS",\n      "meta": {\n        "extension": [ {\n          "url": "https://hapifhir.org/NamingSystem/bulk-export-job-id",\n          "valueString": "d28e0a0a-0b7b-4af9-9fca-23e2eab65d99"\n        }, {\n          "url": "https://hapifhir.org/NamingSystem/bulk-export-binary-resource-type",\n          "valueString": "Immunization"\n        } ],\n        "versionId": "1",\n        "lastUpdated": "2024-01-16T13:37:07.728-05:00"\n      },\n      "contentType": "ap

In [143]:
result = session.get(f"https://bulksandbox.pagekite.me/iis/fhir/Connectathon/Group/100/$export", headers={"Authorization": f"Bearer {token}", "prefer": "respond-async"}, 
                    #   json={
    # "resourceType": "Parameters",
    # "parameter": [
    #     {
    #         "name": "patientReference",
    #         "valueReference": {
    #             "reference" : "Patient/356",
    #             "type" : "Patient"
    #         }
    #     }
    # ]
# }
)
print(result.text)
print(result.headers)
checkurl=  result.headers["Content-Location"]


{'X-Powered-By': 'HAPI FHIR 6.8.3-SUB REST Server (FHIR Server; FHIR 4.0.1/R4)', 'Content-Location': 'http://bulksandbox.pagekite.me/iis/fhir/Connectathon/$export-poll-status?_jobId=ed23d806-c0d1-4b4e-bdee-e6f70a8ad453', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Transfer-Encoding': 'chunked', 'Date': 'Wed, 17 Jan 2024 19:38:17 GMT', 'Keep-Alive': 'timeout=20', 'Connection': 'keep-alive'}


In [144]:
check = session.get(checkurl, headers={"Authorization": f"Bearer {token}"},)
print(check.text)
print(check.content)
print(check.status_code)
print(check.headers)


b''
202
{'X-Powered-By': 'HAPI FHIR 6.8.3-SUB REST Server (FHIR Server; FHIR 4.0.1/R4)', 'X-Progress': 'Build in progress - Status set to IN_PROGRESS at 2024-01-17T14:38:17.508-05:00', 'Retry-After': '120', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Transfer-Encoding': 'chunked', 'Date': 'Wed, 17 Jan 2024 19:38:19 GMT', 'Keep-Alive': 'timeout=20', 'Connection': 'keep-alive'}


In [ ]:
ehrUrl = "http://localhost:9091/ehr"
ehrRes = session.get(ehrUrl + "/smart-test/" + key.get("kid"))
print(ehrRes)